In [12]:
#This uses geopy36
import geopandas as gpd
import pandas as pd
%matplotlib inline  
from shapely.geometry import Point

from simpledbf import Dbf5
#pip install simpledbf
#https://pypi.python.org/pypi/simpledbf/0.2.4

vta_taz = gpd.read_file('/Users/vivek/Github/VTA/Landuse/VTA_TAZ/')
vta_taz = vta_taz.rename(columns={'TAZ':'VTA_TAZ'})

dbf = Dbf5('/Users/vivek/Github/VTA/Landuse/2017ABAGLanduseAllocation/VTA/zmast13.dbf')
vta_dbf = dbf.to_dataframe()
vta_dbf = vta_dbf.rename(columns={'ZONE':'VTA_TAZ'})

vta_taz = pd.merge(vta_taz,vta_dbf)


# mtc_taz = gpd.read_file('/Users/vivek/Github/VTA/Landuse/MTC/MTCTAZ1454.dbf')


#http://analytics.mtc.ca.gov/foswiki/UrbanSimTwo/OutputToTravelModel

In [13]:
mtc_taz = gpd.read_file('/Users/vivek/Github/VTA/Landuse/MTC/')
mtc_taz = mtc_taz.rename(columns={'TAZ1454':'ZONE'})
abag_2010 = pd.read_csv('/Users/vivek/Github/VTA/Landuse/2017ABAGLanduseAllocation/ABAGPROJ2017/DRAFT/run7224c_taz_summaries_2010.csv')
mtc_taz = pd.merge(mtc_taz,abag_2010)

In [14]:
# vta_taz['centroid'] = vta_taz.centroid
# vta_taz = vta_taz.set_geometry('centroid')
# vta_taz['old_geometry'] = vta_taz['geometry']
# vta_taz['geometry'] = vta_taz['centroid']

# centroid = gpd.sjoin(mtc_taz[['ZONE','geometry']], vta_taz, how = "right", op='contains')
# centroid['ZONE'].nunique()
# centroid[['ZONE','VTA_TAZ']].to_csv("rel.csv")
# rel = centroid[['ZONE','VTA_TAZ']]

In [15]:
# vt = vta_taz.columns.values
# ab = abag_2010.columns.values 
# set(ab) - set(tz)
# # set(ab) - set(tz)
# # set(ab).difference(set(tz))
# set(vt).intersection(set(ab))
# set(ab) - set(vt)

In [16]:
rel = pd.read_csv('rel.csv')

In [17]:
rel = rel.loc[~rel['ZONE'].isnull(),]

In [18]:
abag_2010.columns

Index(['zone_id', 'SD', 'ZONE', 'COUNTY', 'AGREMPN', 'FPSEMPN', 'HEREMPN',
       'RETEMPN', 'TOTEMP', 'MWTEMPN', 'OTHEMPN', 'HHINCQ1', 'HHINCQ2',
       'HHINCQ3', 'HHINCQ4', 'SHPOP62P', 'TOTHH', 'GQPOP', 'MFDU', 'SFDU',
       'CIACRE_UNWEIGHTED', 'RESACRE_UNWEIGHTED', 'CIACRE', 'RESACRE',
       'TOTACRE', 'HHPOP', 'TOTPOP', 'EMPRES', 'AGE0004', 'AGE0519', 'AGE2044',
       'AGE4564', 'AGE65P', 'total_job_spaces', 'total_residential_units'],
      dtype='object')

In [19]:
vt_calc = pd.merge(pd.merge(abag_2010[['ZONE','TOTHH','RESACRE','CIACRE','TOTEMP']],
                            rel,how='outer'),vta_taz[['VTA_TAZ','DIST','SDIST','CITY','COUNTY','geometry','TOTHH','TOTPOP',
                            'HHPOP','EMPRES','HH1','HH2','HH3','HH4','TACRES','RESACRE','CIACRE','TEMP',
                            'RETEMP','SEREMP','OTHEMP','AGEMP','MANEMP','WHOEMP','AGE0004','AGE0519',
                            'AGE2044','AGE4564','AGE65','SFHH','MFHH']].rename(columns={"TOTHH":"VTA_TOTHH","TOTPOP":"VTA_TOTPOP",
                            "HHPOP":"VTA_HHPOP","EMPRES":"VTA_EMPRES","RESACRE":'VTA_RESACRE',"CIACRE":"VTA_CIACRE","TEMP":"VTA_TEMP"}))

In [20]:
vt_calc.head()
# vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_TOTPOP'].apply(lambda x: x / x.sum())
# vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_TOTPOP'].sum()

,ZONE,TOTHH,RESACRE,CIACRE,TOTEMP,index_right,VTA_TAZ,DIST,SDIST,COUNTY,...,AGEMP,MANEMP,WHOEMP,AGE0004,AGE0519,AGE2044,AGE4564,AGE65,SFHH,MFHH
0,1,43.0,1.0,15.0,21892.0,1050.0,1701.0,1,1,1,...,36,269,374,1,0,34,34,10,1,44
1,2,131.0,1.0,24.0,33612.0,1051.0,1702.0,1,1,1,...,55,422,587,15,26,81,93,46,3,145
2,3,254.0,1.0,2.0,2003.0,1052.0,1703.0,1,1,1,...,4,34,47,12,16,193,140,111,6,262
3,4,131.0,1.0,18.0,17831.0,1053.0,1704.0,1,1,1,...,42,324,451,4,0,28,62,41,2,74
4,5,560.0,1.0,15.0,13388.0,1054.0,1705.0,1,1,1,...,37,282,392,16,59,432,314,89,11,506


In [21]:
vt_calc[1000:1200][['ZONE','VTA_TAZ','TOTEMP','VTA_TEMP']].head()

,ZONE,VTA_TAZ,TOTEMP,VTA_TEMP
1000,506,7.0,1927.0,1671
1001,506,16.0,1927.0,562
1002,507,229.0,1114.0,198
1003,507,230.0,1114.0,818
1004,507,18.0,1114.0,270


In [22]:
vt_tothh = vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_TOTHH'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()

vt_resacre = vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_RESACRE'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
vt_ciacre = vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_CIACRE'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
vt_temp = vt_calc.groupby(['ZONE','VTA_TAZ'])['VTA_TEMP'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()

In [23]:
vt_tothh = vt_tothh.rename(columns={"VTA_TOTHH":"VTA_TOTHH_share"})

vt_resacre = vt_resacre.rename(columns={"VTA_RESACRE":"VTA_RESACRE_share"})
vt_ciacre = vt_ciacre.rename(columns={"VTA_CIACRE":"VTA_CIACRE_share"})
vt_temp = vt_temp.rename(columns={"VTA_TEMP":"VTA_TEMP_share"})

In [24]:
vta_final= pd.merge(pd.merge(pd.merge(pd.merge(vt_calc,vt_tothh),vt_resacre),vt_ciacre),vt_temp)

In [25]:
vta_final['abag_2010_TOTHH_dist'] = round(vta_final['TOTHH']*vta_final['VTA_TOTHH_share'])

vta_final['abag_2010_RESACRE_dist'] = round(vta_final['RESACRE']*vta_final['VTA_RESACRE_share'])
vta_final['abag_2010_CIACRE_dist'] = round(vta_final['CIACRE']*vta_final['VTA_CIACRE_share'])
vta_final['abag_2010_TEMP_dist'] = round(vta_final['TOTEMP']*vta_final['VTA_TEMP_share'])

In [26]:
vta_final['abag_2010_HHPOP_dist'] = round(vta_final['VTA_HHPOP']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])

In [27]:
vta_final['abag_2010_TOTPOP_dist'] = round(vta_final['VTA_TOTPOP']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])

In [28]:
vta_final['abag_2010_EMPRES_dist'] = round(vta_final['VTA_EMPRES']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])

In [29]:
vta_final['abag_2010_HH1_dist'] = round(vta_final['HH1']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])
vta_final['abag_2010_HH2_dist'] = round(vta_final['HH2']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])
vta_final['abag_2010_HH3_dist'] = round(vta_final['HH3']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])
vta_final['abag_2010_HH4_dist'] = round(vta_final['HH4']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])


In [30]:
vta_final['abag_2010_RETEMP_dist'] = round(vta_final['RETEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_SEREMP_dist'] = round(vta_final['SEREMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_OTHEMP_dist'] = round(vta_final['OTHEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_AGEMP_dist'] = round(vta_final['AGEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_MANEMP_dist'] = round(vta_final['MANEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])
vta_final['abag_2010_WHOEMP_dist'] = round(vta_final['WHOEMP']/vta_final['VTA_TEMP']*vta_final['abag_2010_TEMP_dist'])

In [31]:
vta_final['abag_2010_AGE0004_dist'] = round(vta_final['AGE0004']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])
vta_final['abag_2010_AGE0519_dist'] = round(vta_final['AGE0519']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])
vta_final['abag_2010_AGE2044_dist'] = round(vta_final['AGE2044']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])
vta_final['abag_2010_AGE4564_dist'] = round(vta_final['AGE4564']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])
vta_final['abag_2010_AGE65_dist'] = round(vta_final['AGE65']/vta_final['VTA_HHPOP']*vta_final['abag_2010_HHPOP_dist'])

In [32]:
vta_final['abag_2010_SFHH_dist'] = round(vta_final['SFHH']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])
vta_final['abag_2010_MFHH_dist'] = round(vta_final['MFHH']/vta_final['VTA_TOTHH']*vta_final['abag_2010_TOTHH_dist'])

In [33]:
vta_final

,ZONE,TOTHH,RESACRE,CIACRE,TOTEMP,index_right,VTA_TAZ,DIST,SDIST,COUNTY,...,abag_2010_AGEMP_dist,abag_2010_MANEMP_dist,abag_2010_WHOEMP_dist,abag_2010_AGE0004_dist,abag_2010_AGE0519_dist,abag_2010_AGE2044_dist,abag_2010_AGE4564_dist,abag_2010_AGE65_dist,abag_2010_SFHH_dist,abag_2010_MFHH_dist
0,1,43.0,1.000000,15.000000,21892.0,1050.0,1701.0,1,1,1,...,48.0,361.0,501.0,1.0,0.0,33.0,33.0,10.0,1.0,42.0
1,2,131.0,1.000000,24.000000,33612.0,1051.0,1702.0,1,1,1,...,72.0,553.0,769.0,13.0,23.0,72.0,82.0,41.0,3.0,128.0
2,3,254.0,1.000000,2.000000,2003.0,1052.0,1703.0,1,1,1,...,4.0,33.0,46.0,11.0,15.0,183.0,133.0,105.0,6.0,248.0
3,4,131.0,1.000000,18.000000,17831.0,1053.0,1704.0,1,1,1,...,38.0,294.0,409.0,7.0,0.0,48.0,107.0,71.0,3.0,128.0
4,5,560.0,1.000000,15.000000,13388.0,1054.0,1705.0,1,1,1,...,29.0,221.0,307.0,17.0,64.0,468.0,340.0,96.0,12.0,548.0
5,6,2162.0,7.000000,8.000000,3417.0,579.0,1706.0,1,1,1,...,3.0,24.0,10.0,48.0,456.0,2350.0,609.0,311.0,13.0,2149.0
6,7,3651.0,13.000000,14.000000,11019.0,581.0,1707.0,1,1,1,...,2.0,349.0,244.0,121.0,458.0,2640.0,1737.0,843.0,7.0,3644.0
7,8,4463.0,8.330422,15.000000,3949.0,583.0,1708.0,1,1,1,...,2.0,52.0,41.0,308.0,698.0,2888.0,3709.0,1796.0,17.0,4446.0
8,9,4932.0,9.275070,47.000000,25650.0,582.0,1709.0,1,1,1,...,3.0,733.0,415.0,338.0,862.0,4195.0,2511.0,1104.0,31.0,4901.0
9,10,3207.0,9.342438,35.000000,9072.0,1055.0,1710.0,1,1,1,...,1.0,482.0,241.0,260.0,573.0,2679.0,1717.0,441.0,77.0,3130.0


In [34]:
vta_final.to_csv('vta_final_abag_2010.csv',index=False)

In [35]:
geometry = vta_final['geometry']
crs = {'init': 'epsg:4326'}
geo_df = gpd.GeoDataFrame(vta_final, crs=crs, geometry=geometry)

In [36]:
geo_df.head()
geo_df.to_file('abag_2010_vta_dist')